# Development Tools

Python provides modules which help us in writing softwares:
- `pydoc` for documentation
- `doctest` & `unittest` for unit tests
- `typing` for type hints

## 1. Typing

`typing` module provides runtime support for type hints in various scenarios.

### 1.1. Type aliases

A type alias is defined by assigning type to the alias.

In [2]:
Vector = list[float]

def alpha(scalar: float, vector: Vector, built: str) -> Vector:
    # do something
    return vector

In above case  `Vector` and `list[float]` will be treated as interchangeable synonyms.

Type aliases are useful for simplifying the complex type signatures.
```
fom collections.abc import Sequence

ConnectionOptions = dict[str, str]
Address = tuple[str, int]

Server = tuple[Address, ConnectionOptions]

def broadcast_message(message: str, servers: Sequence[Server])-> None:
     # some code


# This above mentioned function without Type alias would look like this

def broadcast_message(message: str, servers: Sequence[tuple[tuple[str, int], dict[str, str]]]) -> None:
    # some code
```

Type aliases may be marked with `TypeAlias` to make it explicit that the statement is a type alias declaration, not a normal variable assignment.

```
from typing import TypeAlias

Vector: TypeAlias = list[float]
```

### 1.2. NewType

New type create distinct types.

```
from typing import NewType

UserId = NewType('UserId', int)

id1 = UserId(111111)
```

The static type checker will treat the new type as if it were a subclass of the original type. This is useful in helping catch logical errors:

```
def get_user_name(user_id: UserId) -> str:
    ...

# passes type checking
user_a = get_user_name(UserId(42351))

# fails type checking; an int is not a UserId
user_b = get_user_name(-1)
```

We may still perform all `int` operations on a variable of type `UserId`, but the result will always be of type `int`. This lets you pass in a `UserId` wherever an `int` might be expected, but will prevent you from accidentally creating a `UserId` in an invalid way.

```
# 'output' is of type 'int', not 'UserId'
output = UserId(23413) + UserId(54341)
```

These checks are enforced only by the static type checker. At runtime, the statement `Derived = NewType('Derived', Base)` will make `Derived` a callable that immediately returns whatever parameter you pass it. That means the expression `Derived(some_value)` does not create a new class or introduce much overhead beyond that of a regular function call.

More precisely, the expression `some_value is Derived(some_value)` is always true at runtime.

It is invalid to create a subtype of Derived:

```
from typing import NewType

UserId = NewType('UserId', int)

# Fails at runtime and does not pass type checking
class AdminUserId(UserId): pass
```

However, it is possible to create a NewType based on a ‘derived’ NewType:

```
from typing import NewType

UserId = NewType('UserId', int)

ProUserId = NewType('ProUserId', UserId)
```

### 1.3. Annotating Callable Objects

All callable objects can annotated using `collections.abc.Callable` or `typing.Callable.Callable[[argument list], return variable]` .

Ex - `.Callable[[int], str]`signifies a function that takes a single parameter of type `int` and returns a `str`.

In [3]:
from collections.abc import Callable, Awaitable

def feeder(get_next_item: Callable[[], str]) -> None:
    ...  # Body

def async_query(on_success: Callable[[int], None],
                on_error: Callable[[int, Exception], None]) -> None:
    pass

async def on_update(value: str) -> None:
    pass

callback: Callable[[str], Awaitable[None]] = on_update

The argument list must be a list of types. The return type must be a single type.

If a literal ellipsis `...` is given as the argument list, it indicates that a callable with any arbitrary parameter list would be acceptable.

In [4]:
def concat(x: str, y: str)->str:
    return x + y

x: Callable[..., str]
x = str
x = concat

`Callable` can't express complex signatures, so to do that we define these complex signatures in `__call__()` method of a custom class which inherits from the `Protocol` class.

In [10]:
from collections.abc import Iterable
from typing import Protocol

class Combiner(Protocol):
    def __call__(self, *vals: bytes, maxlen: int ) -> list[bytes]: ...

def batch_proc(data: Iterable[bytes], cb_results: Combiner) -> bytes:
    for item in data:
        ...

def good_cb(*vals: bytes, maxlen: int ) -> list[bytes]:
    ...
def bad_cb(*vals: bytes, maxitems: int) -> list[bytes]:
    ...

batch_proc([], good_cb)  

### 1.4. Generics

Type information about objects kept in containers cannot be inferred in a generic way.

Python provides some container classes that help use to extract those information.

Here we can observe that `Sequence[Car]` indicates that all elements in the sequence must be instance of `Car` class

In [ ]:
from collections.abc import Sequence

class Car: ...

def car_data(car: Sequence[Car], name: str)-> None:...

We can make annotations to objects without using a class also.

In [12]:
from typing import TypeVar
from collections.abc import Sequence

C = TypeVar('C')

def car_data(car: Sequence[C], name: str)-> None:...

### 1.5. Any Type

`Any` is a special kind of type.

A static type checker will treat ever type as being compatible with `Any` and `Any` as being compatible with every type.

Thus it is possible to perform any operation or method call on a value of type `Any` and assign it to any variable.

In [13]:
from typing import Any

# declaring Any type variable
a: Any = None

# Any can be any type of variable 
a = []

a = ""

a = 10

a = True

In [14]:
# A function can also accept argument of Any type

def func(a: Any)-> None:
    ...

In [16]:
type(a) # latest assigned type is being given to a 

bool

## 1.6. Classes, Functions in Typing

`typing` module has various classes, functions and decorators.


**Classes in Typing**
- `typing.AnyStr` it is used for functions that may accept `str` or `bytes` arguments but can't allow to mix them
  - `AnyStr = TypeVar('AnyStr', str, bytes)`

- `typing.LiteralString` special type that only includes literal string 
  - useful for where we generate tokens or passkeys made up of literals only and are strings, so it makes possible to avoid any no literal string generation by acting as static checker


- `typing.Never` a type that has no members, it can be used to define a function that should never be called, or a function that never returns

Only python 3.11+

In [ ]:
from typing import Never

def alpha(arg: Never)->None:
    ...

- `typing.NoReturn` special type to indicate that function never returns

In [20]:
from typing import NoReturn

def alpha()->NoReturn:
    ...

- `typing.Self` special type to represent current enclosed class

Only python 3.11+

In [ ]:
from typing import Self

class Beta:
    def return_Self(self)-> Self:
        return self
    

class Car:
    def return_car(car)-> Car:
        ...

- `typing.TypeAlias` used to annotate aliases that make use of forward reference
- `typing.Optional` `Optional[X]` is equivalent to `X|None` or `Union[X, None]`

Only supported over 3.10+

In [22]:
from typing import Optional

def func(arg: Optional[int] = None)-> None:
    ...

- `typing.Concatenate` special form for annotating higher order functions
- `typing.Literal` special form for annotating literal types
- `typing.ClassVar` special type construct to mark class variables

In [23]:
from typing import ClassVar

class Robot:
    type: ClassVar[str]
    def __init__(self, ammo, hud) -> None:
        self.ammo = ammo

# ammo is instance variable it changes for each instance while hud remains same for all instances throughout the class

- `typing.Final` special type to indicate final names tp type checkers

In [24]:
from typing import Final

HEIGHT_WIN: Final = 720

class Layout:
    Dimension: Final[tuple] = (1080, 720)

- `typing.Required` special type construct to mark a `TypedDict` as required
- `typing.NotRequired` special type construct to mark a `TypedDict` keys as potentially missing
- `typing.Annotated` special type construct to add context-specific metadata to an annotation
    - like this: `Annotated[type, metadata]`

In [27]:
from typing import Annotated
from dataclasses import dataclass

@dataclass
class Values:
    low: int
    high: int

T1 = Annotated[int, Values(0, 10)]
T1 = Annotated[int, Values(100, 1000)]

- `typing.TypeGuard` special type construct for making user-defined type guard functions
  - it can be used to annotate the return type of user defined type guard function
  - it only accepts a single type argument 
  - it aims to benefit type narrowing

In [29]:
def is_str(val: str or float):
    # isinstance a type guard
    if isinstance(val, str):
        # type of val is narrowed to str
        ...
    else: 
        # type of val is narrowed to float
        ...

In [ ]:
# using typeguard

from typing import TypeGuard

def is_str_list(val: list[object]) -> TypeGuard[list[str]]:
    '''Determines whether all objects in the list are strings'''
    return all(isinstance(x, str) for x in val)

- `typing.Unpack` special constructor to mark an object as being unpacked
- `typing.Generic` abstract base class for generic type
- `typing.TypeVar` type variable
- `typing.TypeVarTuple` type variable tuple
- `typing.PramSpec` parameter specification variable
- `typing.PramSpecArgs` parameter specification of arguments
- `typing.PramSpecKwargs` parameter specification of keyword arguments
- `typing.NamedTuple` `typing` version of `collections.namedTuple()`
- `typing.NewType` class to create lower head
- `typing.Protocol` base class for Protocol
- `typing.TypedDict` special construct to add type hints to a dictionary

**Functions in Typing**
- `typing.cast(typ, val)` cast a value to a type
- `typing.assert_type(val, typ. /)` ask a static type checker to confirm that `val` has an inferred type of `typ`
- `typing.assert_never(arg, /)` ask a static type checker to confirm that a line of code is unreachable
- `typing.reveal_type(obj, /)` reveal the inferred static type of an expression

## 2. Pydoc

`pydoc` module automatically generates documentation from python modules.

This documentation can be presented as:
- text on console
- save as HTML pages
- served to browser

For any module, class, functions, attributes documentation is derived from the docstring `__doc__` attribute of the object and recursively of its documentable members.

In case their is no docstring, `pydoc` tries to obtain description from the block of the comment lines just above the definition of the class, function or method source file or at the top of the module.

The built-in function help() invokes the online help system in the interactive interpreter, which uses `pydoc` to generate its documentation as text on the console.

To view a list of all possible options associated with `pydoc` and generate documentation use this command:
``` 
python -m pydoc
```

In [32]:
__doc__

'Automatically created module for IPython interactive environment'

In [34]:
!python -m pydoc

pydoc - the Python documentation tool

pydoc <name> ...
    Show text documentation on something.  <name> may be the name of a
    Python keyword, topic, function, module, or package, or a dotted
    reference to a class or function within a module or module in a
    package.  If <name> contains a '\', it is used as the path to a
    Python source file to document. If name is 'keywords', 'topics',
    or 'modules', a listing of these things is displayed.

pydoc -k <keyword>
    Search for a keyword in the synopsis lines of all available modules.

pydoc -n <hostname>
    Start an HTTP server with the given hostname (default: localhost).

pydoc -p <port>
    Start an HTTP server on the given port on the local machine.  Port
    number 0 can be used to get an arbitrary unused port.

pydoc -b
    Start an HTTP server on an arbitrary unused port and open a Web browser
    to interactively browse documentation.  This option can be used in
    combination with -n and/or -p.

pydoc -w <name> .

This will load all documentation of `module` in command prompt

In [ ]:
!python -m pydoc numpy

To generate documentation about custom documentation use
```
python -m pydoc -w my_module
```

## 3. Python Development Mode

When development mode is activated additional runtime checks are enabled.

Development mode is enabled using:
- `-X dev` command line option or
- by setting `PYTHONDEVMODE` environment variable to `1`

Enabling development environment is similar to the following command:
```
PYTHONMALLOC=debug PYTHONASYNCIODEBUG=1 python3 -W default -X faulthandler script.py
```
**Effects of Development Mode**
- Warnings are shown only if issue is detected else not due to `warning filter` [its done by providing`-W`]
- Debug hooks on memory to check for [its done by `PYTHONMALLOC=debug`]:
  - buffer overflow and underflow
  - Memory allocator API violation
  - Unsafe usage of the GIL

- `faulthandler.enable()` is called on startup to dump python traceback on crash [done by `-X faulthandler`]
- asyncio debug mode is activated, it checks for coroutines that were not awaited and logs them [its done by `PYTHONASYNCIODEBUG=1`]

## 4. Doctest

`doctest` module searches for part of text in python comments that are interactive and then executes them to verify that they work/

**Use cases**
- To verify that the modules docstring's are up-to-date and examples provided in them are still working as documented.
- To perform regression testing by verifying that interactive examples from test file or test object work as expected.
- To write tutorial documentation for modules with interactive examples.

**Using Doctest**
- To use docstring's in a module we have to add,
```
if __name__ == "__main__":
    import doctest
    doctest.testmod()
```

this piece of code at end of module(say module is `M.py`)

- Then we can run the module as script to see the `doctest` in action.
```
python M.py
```

- This will return nothing in output and that means all examples are working fine. To get the example as output in console add the verbose argument in the previous command.
```
python M.py -v
```

- We can also force verbose even in the module code so later we don't have to pass it.
```
if __name__ = "__main__":
    import doctest
    doctest.testmod(verbose=True)
```

- We can also run `doctest` directly from command line without adding any extra code in the module itself using following command.
```
python -m doctest -v M.py
```

**Using Doctest with Text files**
- We can use `doctest` to test examples from a separate text file using the `testfile()` function.
```
if __name__ == "__main__":
    import doctest
    doctest.tetfile('t.txt')
```

**API in Doctest**
Their are three API available in `doctest`
- Basic API
- Unitest API
- Advance API
  
**Basic API**
It has three available methods:
- `doctest.testmod()`
- `doctest.testfile()`
- `doctest.run_docstring_examples()`

**Unitest API**
It has following methods available:
- `doctest.DocFileSuite()`
- `doctest.DocTestSuite()`
- `doctest.set_unitest_reportflags()`
  
**Advance API**
Its basically backend of the wrappers of all methods in Basic API, it has following classes available:
- `class doctest.DocTest()`
- `class doctest.Example()`
- `class doctest.DocTestFinder()`
- `class doctest.DocTestParser()`
- `class doctest.DocTestRunner()`
- `class doctest.OutputChecker`

**Note:** 
- `testfile(), testmod()` will not show anything in output console until examples fail to execute or `verbose=True` or `-v` argument passed in command line.

- More about `doctest` working in background and API [doctesst](https://docs.python.org/3/library/doctest.html)

This code below would be in separate script name `M.py` and then this script will act as module and we can then use all the above mentioned `doctest` commands and code over it.

In [39]:
"""
This is "M" module. It contains various functions to do mathematical calculations.

For example:

>> add(2, 5)
7
"""

def add(a, b):
    """Return sum of a and b
    >>add(5, 10)
    15
    >>add(5, 0)
    5
    >>add(-2, 5)
    3
    """
    return a + b

This is a sample text file (in this case `t.txt`) which can be used with `doctest` to test examples.

```
The ``M`` module
====================

Using  ``add``

This is an file containing examples in reStructuredText format.

First import ``add`` from ``M`` module:
    >> from M import add

Now use it:
    >> add(10, 5)
    15
```

## 5. Unittest

Unit testing is a software testing technique where individual components or units of a program are tested in isolation to ensure they function correctly and produce the expected results. It helps identify bugs early in the development process and contributes to the overall stability and reliability of the software.

**Type of Unittesting**

There are several types of unit testing:

1. White Box Testing: This type of testing involves examining the internal logic and structure of the code being tested. Test cases are designed based on the code's internal pathways, conditions, and branches.

2. Black Box Testing: Here, the focus is on testing the external behavior of the code. Test cases are designed based on the expected inputs and outputs, without considering the internal implementation.

3. Gray Box Testing: A combination of white box and black box testing, where some knowledge of the internal code is used to design test cases.

4. Positive Testing: Testing where valid inputs are provided to the unit, expecting valid outputs.

5. Negative Testing: Testing where invalid or unexpected inputs are provided to the unit, testing how it handles errors.

6. Boundary Testing: Testing where the unit is tested with inputs at the extreme boundaries of valid ranges, to check how it handles edge cases.

7. Stub Testing: Involves creating simple, partial implementations of components to test a specific piece of code.

8. Mock Testing: Similar to stub testing, but with more control over the simulated behavior of components.

9. Isolation Testing: Ensures that a specific unit is isolated and tested independently from the rest of the application.

10. Component Testing: Involves testing individual components or modules of the software in isolation.

11. Integration Testing: While not exactly unit testing, it involves testing the interaction between different components or modules to ensure they work well together.

These types of unit testing help ensure different aspects of a software component's functionality and behavior are thoroughly tested.

`unittest` is a unit testing framework is used for performing unit testing in python.

It supports:
- Test automation
- Sharing of setup and shutdown code for 
  - tests
  - aggregation of tests into collections
  - independence of the tests from the reporting framework

To achieve this, unittest supports some important concepts in an object-oriented way:
- *test fixture* A test fixture represents the preparation needed to perform one or more tests, and any associated cleanup actions. This may involve, for example, creating temporary or proxy databases, directories, or starting a server process.

- *test case* A test case is the individual unit of testing. It checks for a specific response to a particular set of inputs. unittest provides a base class, TestCase, which may be used to create new test cases.

- *test suite* A test suite is a collection of test cases, test suites, or both. It is used to aggregate tests that should be executed together.

- *test runner*A test runner is a component which orchestrates the execution of tests and provides the outcome to the user. The runner may use a graphical interface, a textual interface, or return a special value to indicate the results of executing the tests.

`unitest` follows a general procedure to  perform testing which includes:
1. A class is created for a test case by sub classing `unittest.TestCase`.
2. Various tests are defined in the form of methods within this class for particular response with various set of inputs. Name of these methods must start with `test` word just as convention to inform the test runner abut which methods represent tests.
3. Each test method has various `asssert` methods to check and report failures.

```
import unittest

class TestClass(unittest.TestCase):
    def test_something(self):
        self.assert_smthng(inputs)
    
    def test_something_else(self):
        self.assert_smthng(inputs)

if __name__ == "__main__":
    unittest.main()
```

**Executing Testing**
- This piece of code `unittest.main()` provides a command line interface to test the script.

- To test the scrip using command line we have do following:
```
python -m unittest test_module.TestClass

# OR

python -m unittest test_module.TestClass.test_method

# OR

python -m unittest [test module1, test module 2, ...].TestClass
```

- Test can be performed by specifying file path as well:
```
python -m  unittest tests/test_smthng.py
```

In this case path is converted to module name by removing `.py` extension and it is compulsory that script is importable.

- To get more details about testing in console we may turn verbose True in script method as `unittest.main(verbose=True)` or we may pass `-v` argument with any command in command line.

**Command Line Options for Running test Command**
- `-v` verbose to get more information in command line
- `-h` for list of all command line options, i.e help
- `-b` or `--buffer` standard output and error streams are buffered during the test run. Output during a passing test is discarded. Output is echoed normally on test fail or error and is added to the failure messages
- `-c` or `--catch` first control-c during the test run waits for the current test to end and then reports all the results so far. A second control-c raises the normal KeyboardInterrupt exception
- `f` or `--failfast` stop the test run on the first error
- `--locals` show local variables in tracebacks
- `k` only run test methods and classes that match the pattern or substring. 
  - This option may be used multiple times, in which case all test cases that match any of the given patterns are included.
  - Patterns that contain a wildcard character (*) are matched against the test name using fnmatch.fnmatchcase(); otherwise simple case-sensitive substring matching is used.
  - Patterns are matched against the fully qualified test method name as imported by the test loader
  - For example, `-k foo` matches `foo_tests.SomeTest.test_something, bar_tests.SomeTest.test_foo`, but not` bar_tests.FooTest.test_something`


**All available assert methods in assert**
|Method| Checks that|
|-----------|------------|
|`assertEqual(a, b)`|a == b|
|`assertNotEqual(a, b)`|a != b|
|`assertTrue(x)`|bool(x) is True|
|`assertFalse(x)`|bool(x) is False|
|`assertFalse(x)`|a is b|
|`assertIsNot(a, b)`|a is not b|
|`assertIsNot(a, b)`|x is None|
|`assertIsNot(a, b)`|x is not None|
|`assertIsNot(a, b)`|a in b|
|`assertIsNot(a, b)`|a not in b|
|`assertIsNot(a, b)`|isinstance(a, b)|
|`assertIsNot(a, b)`|not isinstance(a, b)|
|`assertAlmostEqual(a, b)`|round(a-b, 7) == 0|
|`assertNotAlmostEqual(a, b)`|round(a-b, 7) != 0|
|`assertGreater(a, b)`|a > b|
|`assertGreaterEqual(a, b)`|a >= b|
|`assertLess(a, b)`|a < b|
|`assertLessEqual(a, b)`|a <= b|
|`assertRegex(s, r)`|r.search(s)|
|`assertNotRegex(s, r)`|not r.search(s)|
|`assertCountEqual(a, b)`|a & b have the same elements in same length|
|`assertMultiLineEqual(a, b)`|use to compare strings|
|`assertMultiLineEqual(a, b)`|use to compare sequences|
|`assertListEqual(a, b)`|use to compare lists|
|`assertTupleEqual(a, b)`|use to compare tuples|
|`assertSetEqual(a, b)`|use to compare sets or frozensets|
|`assertDictEqual(a, b)`|use to compare dicts|

In [ ]:
from typing import Final

HEIGHT_WIN: Final = 720

class Layout:
    Dimension: Final[tuple] = (1080, 720)

In [ ]:
# Example of Unittest class (almost everything tha can be done with this module)
import unittest

class TestStringMethods(unittest.TestCase):

    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # check that s.split fails when the separator is not a string
        with self.assertRaises(TypeError):
            s.split(2)

if __name__ == '__main__':
    unittest.main()

**Test Discovery**
`unittest` supports test discovery. In order for this to function all tests files must be modules or packages and importable from the top-level directory.

**Executing Discovery**
- This service is provided by `TestLoader.discover()` method that can be added to the module.
- To discover test using command line we have to do this:
```
# in the project directory
python -m unittest discover
```

**Command line options for discovering tests**
- `-v` or `--verbose` verbose to get more information in command line
- `-s`, `--start-directory` directory directory to start discovery (. default)
- `-p`, `--pattern` pattern pattern to match test files (`test*.py` default)
- `-t`, `--top-level-directory directory` top level directory of project (defaults to start directory)

One thing, `python -m unittest discovery` is the equivalent of `python -m unittest`

Examples- 
```
python -m unittest discover -s project_directory -p "*_test.py"
python -m unittest discover project_directory "*_test.py"
```

**Test Code Organization**
- The basic building blocks of unit testing are test cases — single scenarios that must be set up and checked for correctness. 
- In unittest, test cases are represented by `unittest.TestCase` instances. 
- To make our own test cases we must write subclasses of `TestCase` or use `FunctionTestCase`.
- The testing code of a `TestCase` instance should be entirely self contained, such that it can be run either in isolation or in arbitrary combination with any number of other test cases.
- In order to test something, we use one of the `assert*()` methods provided by the `TestCase` base class. If the test fails, an exception will be raised with an explanatory message, and `unittest` will identify the test case as a failure. Any other exceptions will be treated as errors.
- Tests can be numerous, and their set-up can be repetitive. To make it easier we can factor out set-up code by implementing a method called `setUp()`, which the testing framework will automatically call for every single test we run. It basically allows to perform various tests on same thing again and again with varying input.

```
class TestClass(unittest.TestCase):
    def setUp(self):
        self.object = object_being_tested

    def test_something(self):
        self.assert_smthng(object)
    
    def test_something_else(self):
        self.assert_smthng(object)
```

- A `tearDown()` method is also available that tidies up after the test method has been run.
  
```
class TestClass(unittest.TestCase):
    def setUp(self):
        self.object = object_being_tested

    def test_something(self):
        self.assert_smthng(object)
    
    def test_something_else(self):
        self.assert_smthng(object)
    
    def tearDown(self):
        self.object.disposes()
```

- We can group tests together according to feature they test using `TestSuite`.

```
def suite():
    suite = unittest.TestSuite()
    suite.addTest(TestClass(test_method_one))
    suite.addTest(TestClass(test_method_two))

if __name__ == "__main__":
    runner = unittest.TextTestRunner()
    runner.run(suite())
```

- Definition of tests, i.e `TestClass` and suite can be in same module.


**Note:** Tests are run in the order of the alphabetical arrangement of the name of test methods.

In [ ]:
# Using Setup method to setup an object which is tested with varying parameters in later test methods
import unittest

class WidgetTestCase(unittest.TestCase):
    def setUp(self):
        self.widget = Widget('The widget')

    def test_default_widget_size(self):
        self.assertEqual(self.widget.size(), (50,50),
                         'incorrect default size')

    def test_widget_resize(self):
        self.widget.resize(100,150)
        self.assertEqual(self.widget.size(), (100,150),
                         'wrong size after resize')

In [ ]:
# Using TearDown method to dispose object setup by the Setup method
import unittest

class WidgetTestCase(unittest.TestCase):
    def setUp(self):
        self.widget = Widget('The widget')

    def test_default_widget_size(self):
        self.assertEqual(self.widget.size(), (50,50),
                         'incorrect default size')

    def test_widget_resize(self):
        self.widget.resize(100,150)
        self.assertEqual(self.widget.size(), (100,150),
                         'wrong size after resize')
        
    def tearDown(self):
        self.widget.dispose()

In [ ]:
# TestSuite to group similar feature testing tests

def suite():
    suite = unittest.TestSuite()
    suite.addTest(WidgetTestCase('test_default_widget_size'))
    suite.addTest(WidgetTestCase('test_widget_resize'))
    return suite

if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(suite())

**Note:** 
- For all available classes, methods and attributes refer [uinttest](https://docs.python.org/3/library/unittest.html#assert-methods)

- Their is an extension module associated with `unittest` framework `unittest.mock` [mock object library](https://docs.python.org/3/library/unittest.mock.html)


## 6. Test

- It is meant for internal use by python only!
- For all available classes, methods and attributes refer [test](https://docs.python.org/3/library/test.html)

- Their is an extension module associated with `test` framework `test.support` [support library](https://docs.python.org/3/library/test.html#module-test.support)


